In [1]:
import pandas as pd
import numpy as np

from data_service import (get_shibor_data,get_interpld_shibor,query_china_shibor_all)

from CVIX import (filter_contract,cal_forward_price,cal_forward_price,get_cboe_vix)

In [2]:
opt_data = pd.read_csv('opt_data.csv',index_col=[0])
# df_rate = pd.read_csv('shibor_df.csv',index_col=0) # from jqdata

shibor_df = get_shibor_data('2015-02-09','2022-05-27')
rate_df = get_interpld_shibor(shibor_df)

In [6]:
opt_data['date'].unique()

array(['2015-02-09', '2015-02-10', '2015-02-11', ..., '2022-05-25',
       '2022-05-26', '2022-05-27'], dtype=object)

In [11]:
COND = (opt_data['date'] == '2015-02-09')
filter_df = opt_data[COND]

In [75]:
maturity, jy_matrix, cjy_matrix =  filter_contract(filter_df)

In [68]:
def _get_group_smallest(df:pd.Series,n:int=1)->pd.DataFrame:
    """获取opt_data中maturity中最小的两个值

    Args:
        df (pd.Series)
        | idnex | list_date | exercise_date | exercise_price | contract_type | code          |
        | :---- | :-------- | :------------ | :------------- | :------------ | :------------ |
        | 0     | 2021/7/29 | 2022/3/23     | 4.332          | CO            | 10003549.XSHG |

    Returns:
        pd.DataFrame
    """
    cond = (df['maturity'] <= np.sort(df['maturity'].unique())[n])
    
    return df[cond]

tmp_df = opt_data.groupby('date',group_keys=False).apply(_get_group_smallest)

In [72]:
smallest_maturity:pd.Series = tmp_df.groupby('date').apply(lambda x:np.sort(x['maturity'].unique())[:2])

In [76]:
cjy_matrix

contract_type,call,put,diff
exercise_price,,,
2.20,0.2100,0.0880,0.1220
2.25,0.1785,0.1111,0.0674
2.30,0.1466,0.1327,0.0139
2.35,0.1261,0.1614,0.0353
2.40,0.1008,0.1903,0.0895


In [77]:
tmp_df

,date,exercise_date,close,contract_type,exercise_price,maturity
0,2015-02-09,2015-03-25,0.1826,call,2.200,0.120548
656,2015-02-09,2015-04-22,0.1614,put,2.350,0.197260
609,2015-02-09,2015-04-22,0.1327,put,2.300,0.197260
562,2015-02-09,2015-04-22,0.1111,put,2.250,0.197260
515,2015-02-09,2015-04-22,0.0880,put,2.200,0.197260
...,...,...,...,...,...,...
207889,2022-05-27,2022-06-22,0.3190,put,3.061,0.071233
213693,2022-05-27,2022-06-22,0.2560,put,3.000,0.071233
220175,2022-05-27,2022-06-22,0.2497,call,2.500,0.071233
208030,2022-05-27,2022-06-22,0.4207,put,3.159,0.071233


In [ ]:
# near_term_opt
# next_term_opt